In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
import time, os, fnmatch, shutil

In [ ]:
cmmd_dir = "/media/craig/Larry/python/"

train_datagen = ImageDataGenerator(
    preprocessing_function=keras.applications.resnet50.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(
    preprocessing_function=keras.applications.resnet50.preprocess_input
)
train_generator = train_datagen.flow_from_directory(
        cmmd_dir+"subset_two/PNG/TRAIN/",
        color_mode='rgb',
        target_size=(224, 224),
        batch_size=32,
        shuffle=True,
        class_mode='binary',
        seed=11)
validation_generator = test_datagen.flow_from_directory(
        cmmd_dir+"subset_two/PNG/VAL/",
        color_mode='rgb',
        target_size=(224, 224),
        batch_size=32,
        shuffle=True,
        class_mode='binary',
        seed=11)

# ResNet50 Transfer Learning

In [ ]:
ResNet50_base = tf.keras.applications.resnet50.ResNet50(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

for layer in ResNet50_base.layers:
    layer.trainable = False

In [ ]:
model = tf.keras.models.Sequential()
model.add(ResNet50_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.layers[0].trainable = False


In [ ]:
model.summary()

In [ ]:
import time, os, fnmatch, shutil
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    #TensorBaoard TITLE:
    run_id = ("ResNet50_Transfer_Learning")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [
    keras.callbacks.ModelCheckpoint("/media/craig/Henry/ML_models/resnet50/save_at_{epoch}.h5"),
    keras.callbacks.TensorBoard(run_logdir)
]

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.0001),
#               loss=keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=[keras.metrics.BinaryAccuracy()])
# history = model.fit(train_generator, epochs=500, callbacks=callbacks, validation_data=validation_generator)

In [ ]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[keras.metrics.BinaryAccuracy()]
)

In [ ]:
history = model.fit(
    train_generator,
#     steps_per_epoch=75,
    epochs=500,
    validation_data=validation_generator,
#     validation_steps=25,
    callbacks=callbacks
)

In [ ]:
model.save('/media/craig/Henry/ML_models/resnet50/final_model_epoch_500.h5')

In [ ]:
loss_hist = history.history['val_loss']

lowest_val_loss = np.min(loss_hist)
best_epochs = np.argmin(loss_hist) + 1
print("The lowest validation loss is: "+str(lowest_val_loss)+"\nAt epoch: "+str(best_epochs))

bin_acc_hist = history.history['val_binary_accuracy']

lowest_val_loss = np.max(bin_acc_hist)
best_epochs = np.argmax(bin_acc_hist) + 1
print("The highest validation acc is: "+str(lowest_val_loss)+"\nAt epoch: "+str(best_epochs))